# VDEH Language Detection Pipeline

**Fokus:** Professionelle Sprach-Erkennung mit langdetect (Dual-Source Strategie)

## 🎯 Ziel
- Sprach-Erkennung für alle VDEH Titel mittels langdetect
- Confidence-Scoring und Qualitätsfilter
- Sprachname-Mapping und Kategorisierung
- **Dual-Source Ansatz**: Beibehaltung der MARC21-Sprache + langdetect-Erkennung
- Export der sprach-angereicherten Daten

## 📚 Input/Output
- **Input**: `data/vdeh/processed/02_preprocessed_data.parquet` (enthält `language` aus MARC21)
- **Output**: `data/vdeh/processed/03_language_detected_data.parquet` (+ `detected_language` aus langdetect)

## 📊 Dual-Source Strategie
- **MARC21 `language`**: ISO 639-2 Codes (ger, eng, fre) - 51.8% Abdeckung, 100% Genauigkeit
- **langdetect `detected_language`**: ISO 639-1 Codes (de, en, fr) - Titel-basierte Erkennung
- **Fusion in Notebook 05**: Kombiniert beide Quellen → `language_final`

In [1]:
# 🛠️ SETUP: Initialize notebook environment
import sys
from pathlib import Path

# Add src to path (temporary until utils is imported)
project_root = Path.cwd()
while not (project_root / 'config.yaml').exists() and project_root.parent != project_root:
    project_root = project_root.parent
sys.path.insert(0, str(project_root / 'src'))

# Now use the utility function
from utils.notebook_utils import setup_notebook

project_root, config = setup_notebook()
print(f"✅ Project root: {project_root}")
print(f"✅ Project: {config.get('project.name')} v{config.get('project.version')}")

2025-12-09 05:54:44 - utils.notebook_utils - INFO - Searching for project root...


2025-12-09 05:54:44 - utils.notebook_utils - INFO - Project root found: /media/sz/Data/Bibo/analysis


2025-12-09 05:54:44 - utils.notebook_utils - INFO - Loading configuration...


2025-12-09 05:54:44 - config_loader - INFO - Configuration loaded from /media/sz/Data/Bibo/analysis/config.yaml


2025-12-09 05:54:44 - utils.notebook_utils - INFO - Configuration loaded successfully: Dual-Source Bibliothek Bestandsvergleich


✅ Project root: /media/sz/Data/Bibo/analysis
✅ Project: Dual-Source Bibliothek Bestandsvergleich v2.0.0


In [2]:
# 📂 DATEN AUS VORHERIGER STUFE LADEN
import pandas as pd
import json

processed_dir = config.project_root / config.get('paths.data.vdeh.processed')
input_path = processed_dir / '02_preprocessed_data.parquet'
metadata_path = processed_dir / '02_metadata.json'

if not input_path.exists():
    raise FileNotFoundError(f"Input-Datei nicht gefunden: {input_path}\n"
                          "Bitte führen Sie zuerst 02_vdeh_data_preprocessing.ipynb aus.")

# Daten laden
df_vdeh = pd.read_parquet(input_path)

# Vorherige Metadaten laden
with open(metadata_path, 'r') as f:
    prev_metadata = json.load(f)

print(f"📂 Daten geladen aus: {input_path}")
print(f"📊 Records: {len(df_vdeh):,}")
print(f"📋 Spalten: {list(df_vdeh.columns)}")
print(f"📅 Vorherige Verarbeitung: {prev_metadata['processing_date']}")

# Check for MARC21 language column
if 'language' in df_vdeh.columns:
    marc21_lang_count = df_vdeh['language'].notna().sum()
    print(f"\n🌍 MARC21 Sprache vorhanden: {marc21_lang_count:,} ({marc21_lang_count/len(df_vdeh)*100:.1f}%)")
    print(f"   → Wird beibehalten für Dual-Source Strategie")
else:
    print(f"\n⚠️  Keine MARC21 Sprache in Daten (erwartet ab MARC21 Migration)")

📂 Daten geladen aus: /media/sz/Data/Bibo/analysis/data/vdeh/processed/02_preprocessed_data.parquet
📊 Records: 58,305
📋 Spalten: ['id', 'title', 'authors', 'authors_affiliation', 'year', 'publisher', 'isbn', 'issn', 'pages', 'language', 'authors_str', 'num_authors', 'authors_affiliation_str', 'num_authors_affiliation', 'isbn_valid', 'isbn_status', 'issn_valid', 'issn_status']
📅 Vorherige Verarbeitung: 2025-12-09T05:54:26.003338

🌍 MARC21 Sprache vorhanden: 30,173 (51.8%)
   → Wird beibehalten für Dual-Source Strategie


In [3]:
# 🌍 SPRACH-ERKENNUNG SETUP
import pandas as pd
import numpy as np
import json

# Import language detection and progress bar
from langdetect import detect_langs, LangDetectException
from tqdm import tqdm

print("🌍 === SPRACH-ERKENNUNG SETUP ===")
print("✅ langdetect imported")
print("✅ tqdm imported für Progress-Anzeige")
print("✅ Sprach-Erkennung Setup abgeschlossen")

🌍 === SPRACH-ERKENNUNG SETUP ===
✅ langdetect imported
✅ tqdm imported für Progress-Anzeige
✅ Sprach-Erkennung Setup abgeschlossen


In [4]:
# 🎯 TITEL FILTERN 
print("🔍 Filtere Titel für Spracherkennung...")

# Minimum Textlänge aus Config
min_length = config.get('data_processing.language_detection.min_text_length', 10)

# Leere oder zu kurze Titel filtern
mask = df_vdeh['title'].notna() & (df_vdeh['title'].str.len() >= min_length)
titles_to_process = df_vdeh[mask]

print(f"📊 Ursprüngliche Anzahl Titel: {len(df_vdeh):,}")
print(f"📊 Titel für Spracherkennung: {len(titles_to_process):,}")
print(f"⏭️  Überspringe {len(df_vdeh) - len(titles_to_process):,} leere/zu kurze Titel")

🔍 Filtere Titel für Spracherkennung...
📊 Ursprüngliche Anzahl Titel: 58,305
📊 Titel für Spracherkennung: 56,861
⏭️  Überspringe 1,444 leere/zu kurze Titel


In [5]:
# 🎯 SPRACH-ERKENNUNG FUNKTION
def detect_language_professional(text):
    """
    Professionelle Sprach-Erkennung mit langdetect (konfigurationsbasiert)
    
    Returns:
        tuple: (language_code, confidence, full_language_name)
    """
    min_length = config.get('data_processing.language_detection.min_text_length', 10)
    
    if not text or pd.isna(text) or len(str(text).strip()) < min_length:
        return 'unknown', 0.0, 'Unknown'
    
    try:
        # Text bereinigen
        clean_text = str(text).encode('utf-8', errors='ignore').decode('utf-8').strip()
        
        if len(clean_text) < min_length:
            return 'unknown', 0.0, 'Unknown'
        
        # Sprach-Erkennung
        lang_probs = detect_langs(clean_text)
        best_match = lang_probs[0]
        
        lang_code = best_match.lang
        confidence = round(best_match.prob, 3)
        
        # Standard-Mapping für häufige Sprachen
        supported_langs = {
            'de': 'German',
            'en': 'English', 
            'fr': 'French',
            'es': 'Spanish',
            'it': 'Italian',
            'nl': 'Dutch',
            'pl': 'Polish',
            'ru': 'Russian',
            'ja': 'Japanese',
            'zh': 'Chinese',
            'uk': 'Ukrainian',
            'cs': 'Czech',
            'hu': 'Hungarian',
            'da': 'Danish',
            'fi': 'Finnish',
            'no': 'Norwegian',
            'sv': 'Swedish'  
        }
        
        # Ergänze durch Konfiguration
        config_langs = config.get('data_processing.language_detection.supported_languages', {})
        if isinstance(config_langs, dict):
            supported_langs.update(config_langs)
        elif isinstance(config_langs, list):
            for lang_item in config_langs:
                if isinstance(lang_item, dict):
                    supported_langs.update(lang_item)
        
        # Finde den vollständigen Namen
        full_name = supported_langs.get(lang_code, lang_code.upper())
        
        return lang_code, confidence, full_name
        
    except (LangDetectException, Exception):
        return 'unknown', 0.0, 'Unknown'

print("✅ Sprach-Erkennungsfunktion definiert")
print(f"📏 Min. Textlänge: {config.get('data_processing.language_detection.min_text_length')}")

# Test der Funktion
test_titles = ["Das ist ein deutscher Titel", "This is an English title", "Ceci est un titre français"]
print(f"\n🧪 === FUNKTIONSTEST ===")
for title in test_titles:
    lang_code, confidence, lang_name = detect_language_professional(title)
    print(f"   '{title[:30]}...' → {lang_code} ({confidence:.3f}) - {lang_name}")

✅ Sprach-Erkennungsfunktion definiert
📏 Min. Textlänge: 10

🧪 === FUNKTIONSTEST ===


   'Das ist ein deutscher Titel...' → de (1.000) - German
   'This is an English title...' → en (1.000) - English
   'Ceci est un titre français...' → fr (1.000) - French


In [6]:
# 🌍 SPRACHERKENNUNG DURCHFÜHREN
print("🌍 Führe Spracherkennung durch...")

# Arrays für Ergebnisse initialisieren
language_codes = []
confidence_scores = []
language_names = []

# Progress Bar
with tqdm(titles_to_process['title'], desc="🌍 Sprach-Erkennung") as pbar:
    for title in pbar:
        lang_code, confidence, lang_name = detect_language_professional(title)
        language_codes.append(lang_code)
        confidence_scores.append(confidence)
        language_names.append(lang_name)

print(f"\n✅ Spracherkennung für {len(titles_to_process):,} Titel abgeschlossen")

🌍 Führe Spracherkennung durch...


🌍 Sprach-Erkennung:   0%|          | 0/56861 [00:00<?, ?it/s]

🌍 Sprach-Erkennung:   0%|          | 41/56861 [00:00<02:19, 406.07it/s]

🌍 Sprach-Erkennung:   0%|          | 90/56861 [00:00<02:05, 452.96it/s]

🌍 Sprach-Erkennung:   0%|          | 136/56861 [00:00<02:04, 454.54it/s]

...

🌍 Sprach-Erkennung: 100%|█████████▉| 56702/56861 [02:19<00:00, 395.15it/s]

🌍 Sprach-Erkennung: 100%|█████████▉| 56747/56861 [02:19<00:00, 409.12it/s]

🌍 Sprach-Erkennung: 100%|█████████▉| 56820/56861 [02:19<00:00, 497.19it/s]

🌍 Sprach-Erkennung: 100%|██████████| 56861/56861 [02:19<00:00, 406.17it/s]


✅ Spracherkennung für 56,861 Titel abgeschlossen


In [7]:
# 📊 SPRACH-DATEN ZUM DATAFRAME HINZUFÜGEN
print("📊 === SPRACH-DATEN INTEGRATION ===")

# Neue Spalten initialisieren (mit detected_ Präfix für langdetect-Ergebnisse)
df_vdeh['detected_language'] = 'unknown'
df_vdeh['detected_language_confidence'] = 0.0
df_vdeh['detected_language_name'] = 'Unknown'

# Ergebnisse einfügen (nur für verarbeitete Titel)
df_vdeh.loc[titles_to_process.index, 'detected_language'] = language_codes
df_vdeh.loc[titles_to_process.index, 'detected_language_confidence'] = confidence_scores
df_vdeh.loc[titles_to_process.index, 'detected_language_name'] = language_names

print(f"✅ Sprach-Spalten hinzugefügt (detected_language, detected_language_confidence, detected_language_name)")
print(f"📊 DataFrame: {len(df_vdeh.columns)} Spalten, {len(df_vdeh):,} Zeilen")

# MARC21 language column bleibt erhalten
if 'language' in df_vdeh.columns:
    print(f"✅ MARC21 'language' Spalte beibehalten (Dual-Source Strategie)")

# Qualitäts-Statistiken
min_confidence = config.get('analysis.quality_filters.min_confidence_score', 0.3)
high_confidence_count = sum(1 for c in confidence_scores if c >= min_confidence)
print(f"🎯 Hohe Konfidenz (>={min_confidence}): {high_confidence_count:,} Titel")

# Sprach-Verteilung
lang_dist = pd.Series(language_names).value_counts().head(10)
print(f"\n🌍 Top 10 erkannte Sprachen (langdetect):")
for lang, count in lang_dist.items():
    pct = count/len(language_names)*100 if len(language_names) > 0 else 0
    print(f"   {lang:15}: {count:6,} ({pct:5.1f}%)")

# Vergleich MARC21 vs langdetect (wenn MARC21 vorhanden)
if 'language' in df_vdeh.columns:
    marc21_available = df_vdeh['language'].notna().sum()
    langdetect_available = df_vdeh['detected_language'].ne('unknown').sum()
    both_available = ((df_vdeh['language'].notna()) & (df_vdeh['detected_language'].ne('unknown'))).sum()
    print(f"\n📊 Dual-Source Abdeckung:")
    print(f"   MARC21 Sprache:     {marc21_available:6,} ({marc21_available/len(df_vdeh)*100:.1f}%)")
    print(f"   langdetect Sprache: {langdetect_available:6,} ({langdetect_available/len(df_vdeh)*100:.1f}%)")
    print(f"   Beide vorhanden:    {both_available:6,} ({both_available/len(df_vdeh)*100:.1f}%)")
    print(f"   → Fusion in Notebook 05")

📊 === SPRACH-DATEN INTEGRATION ===
✅ Sprach-Spalten hinzugefügt (detected_language, detected_language_confidence, detected_language_name)
📊 DataFrame: 21 Spalten, 58,305 Zeilen
✅ MARC21 'language' Spalte beibehalten (Dual-Source Strategie)
🎯 Hohe Konfidenz (>=0.3): 56,824 Titel

🌍 Top 10 erkannte Sprachen (langdetect):
   German         : 32,542 ( 57.2%)
   English        : 16,496 ( 29.0%)
   French         :    929 (  1.6%)
   Italian        :    782 (  1.4%)
   SL             :    660 (  1.2%)
   RO             :    629 (  1.1%)
   AF             :    521 (  0.9%)
   CA             :    502 (  0.9%)
   Dutch          :    498 (  0.9%)
   ET             :    396 (  0.7%)

📊 Dual-Source Abdeckung:
   MARC21 Sprache:     30,173 (51.8%)
   langdetect Sprache: 56,859 (97.5%)
   Beide vorhanden:    29,929 (51.3%)
   → Fusion in Notebook 05


In [8]:
# 💾 LANGUAGE-DETECTED DATEN EXPORTIEREN
output_path = processed_dir / '03_language_detected_data.parquet'
df_vdeh.to_parquet(output_path, index=False)

print(f"💾 === LANGUAGE DETECTION ABGESCHLOSSEN ===")
print(f"✅ Language-detected Daten exportiert: {output_path}")
print(f"📊 Records: {len(df_vdeh):,}")
print(f"📋 Spalten: {len(df_vdeh.columns)} (inkl. {len([c for c in df_vdeh.columns if 'lang' in c])} Sprach-Spalten)")
print(f"💾 Dateigröße: {df_vdeh.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Detaillierte Sprach-Statistiken für Metadaten
lang_stats = {
    'total_titles_analyzed': int(len(titles_to_process)),  # titles_to_process statt all_titles
    'high_confidence_count': int(high_confidence_count),
    'confidence_threshold': float(min_confidence),
    'top_languages': {k: int(v) for k,v in dict(lang_dist.head(5)).items()},
    'confidence_distribution': {
        'mean': float(np.mean(confidence_scores)),
        'median': float(np.median(confidence_scores)),
        'std': float(np.std(confidence_scores))
    }
}

💾 === LANGUAGE DETECTION ABGESCHLOSSEN ===
✅ Language-detected Daten exportiert: /media/sz/Data/Bibo/analysis/data/vdeh/processed/03_language_detected_data.parquet
📊 Records: 58,305
📋 Spalten: 21 (inkl. 4 Sprach-Spalten)
💾 Dateigröße: 57.9 MB


In [9]:
# Statistische Auswertung erstellen
from datetime import datetime

lang_stats = {
    'processing_date': datetime.now().isoformat(),
    'num_total': int(len(df_vdeh)),
    'num_processed': int(df_vdeh['detected_language'].notna().sum()),
    'num_errors': int(df_vdeh.get('detect_error', pd.Series(dtype='bool')).notna().sum()),
    'language_analysis': {
        'total_titles_analyzed': int(len(titles_to_process)),
        'high_confidence_count': int(sum(df_vdeh['detected_language_confidence'] >= config.get('data_processing.language_detection.confidence_threshold', 0.5)))
    },
    'lang_distribution': {
        str(lang): int(count.item()) if hasattr(count, 'item') else int(count)  # Konvertiere NumPy Typen
        for lang, count in df_vdeh['detected_language'].value_counts().to_dict().items()
        if not pd.isna(lang)
    },
    'dual_source_stats': {
        'marc21_available': int(df_vdeh['language'].notna().sum()) if 'language' in df_vdeh.columns else 0,
        'langdetect_available': int(df_vdeh['detected_language'].ne('unknown').sum()),
        'both_available': int(((df_vdeh['language'].notna()) & (df_vdeh['detected_language'].ne('unknown'))).sum()) if 'language' in df_vdeh.columns else 0
    }
}

# Metadaten speichern
metadata_path = processed_dir / '03_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(lang_stats, f, indent=2)

print(f"📄 Metadaten gespeichert: {metadata_path}")
print(f"\n➡️  Nächster Schritt: 04_vdeh_data_enrichment.ipynb (DNB Enrichment)")
print(f"➡️  Dual-Source Fusion: 05_vdeh_data_fusion.ipynb")

📄 Metadaten gespeichert: /media/sz/Data/Bibo/analysis/data/vdeh/processed/03_metadata.json

➡️  Nächster Schritt: 04_vdeh_data_enrichment.ipynb (DNB Enrichment)
➡️  Dual-Source Fusion: 05_vdeh_data_fusion.ipynb
